# Générer et lire un fichier texte


r, pour une ouverture en lecture (READ).
w, pour une ouverture en écriture (WRITE), à chaque ouverture le contenu du fichier est écrasé. Si le fichier n'existe pas python le crée.
a, pour une ouverture en mode ajout à la fin du fichier (APPEND). Si le fichier n'existe pas python le crée.
t, pour une ouverture en mode texte.
x, crée un nouveau fichier et l'ouvre pour écriture

fichier.write("\n") #'\n' saut de ligne
fichier.write("1\t2\t3\n")# '\t' tabulation

In [3]:
from pylab import *
import numpy as np


l=10
Nn=500
omeg = 25.
xp = np.linspace(-l, l, Nn)

# définition du potentiel quadratique
Pot = (omeg**2/2)*xp**2
V=str(Pot)


fichier = open("data.txt", "w")
for i in range(Nn):
    print(xp[i], Pot[i],file=fichier)
fichier.close()



In [130]:

with open ("data.txt", "r") as fichier: # ouverture du fichier en mode lecture

    fichier = open("data.txt", "r")
data = np.loadtxt(fichier)

xpread = data[:,0]
potread = data[:,1]

print(xpread)
print(potread)






[-10.          -9.95991984  -9.91983968  -9.87975952  -9.83967936
  -9.7995992   -9.75951904  -9.71943888  -9.67935872  -9.63927856
  -9.5991984   -9.55911824  -9.51903808  -9.47895792  -9.43887776
  -9.3987976   -9.35871743  -9.31863727  -9.27855711  -9.23847695
  -9.19839679  -9.15831663  -9.11823647  -9.07815631  -9.03807615
  -8.99799599  -8.95791583  -8.91783567  -8.87775551  -8.83767535
  -8.79759519  -8.75751503  -8.71743487  -8.67735471  -8.63727455
  -8.59719439  -8.55711423  -8.51703407  -8.47695391  -8.43687375
  -8.39679359  -8.35671343  -8.31663327  -8.27655311  -8.23647295
  -8.19639279  -8.15631263  -8.11623246  -8.0761523   -8.03607214
  -7.99599198  -7.95591182  -7.91583166  -7.8757515   -7.83567134
  -7.79559118  -7.75551102  -7.71543086  -7.6753507   -7.63527054
  -7.59519038  -7.55511022  -7.51503006  -7.4749499   -7.43486974
  -7.39478958  -7.35470942  -7.31462926  -7.2745491   -7.23446894
  -7.19438878  -7.15430862  -7.11422846  -7.0741483   -7.03406814
  -6.99398

Ouverture des fichiers pec_sigmag.txt, pec_sigmau.txt et lecture.

In [6]:

with open ("pec_sigmag.txt", "r") as fichierg: # ouverture du fichier pec_sigmag.txt en mode lecture

    fichierg = open("pec_sigmag.txt", "r")
data = np.loadtxt(fichierg)

xg_read = data[:,0]
potg_read = data[:,1]

print(xg_read)
print(potg_read)

with open ("pec_sigmau.txt", "r") as fichieru: # ouverture du fichier pec_sigmau.txt en mode lecture

    fichieru = open("pec_sigmau.txt", "r")
data_u = np.loadtxt(fichieru)

xu_read = data_u[:,0]
potu_read = data_u[:,1]

print(xu_read)
print(potu_read)



[ 0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8
  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7  2.8  2.9  3.   3.1  3.2
  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1  4.2  4.3  4.4  4.5  4.6
  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5  5.6  5.7  5.8  5.9  6.
  6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9  7.   7.1  7.2  7.3  7.4
  8.   8.1  8.2  8.3  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3
  9.4  9.5  9.6  9.7  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7
 10.9 11.  11.1 11.2 11.3 11.4 11.5 11.7 11.8 11.9 12.  12.1 12.2 12.3
 12.4 12.5 12.6 12.7 12.8 12.9 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7
 13.8 13.9 14.  14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9 15.  15.1
 15.2 15.3 15.4 15.5 15.6 15.7 15.8 15.9 16.  16.1 16.2 16.3 16.4 16.5
 16.6 16.7 16.8 16.9 17.  17.1 17.2 17.3 17.4 17.5 17.6 17.7 17.8 17.9
 18.  18.1 18.2 18.3 18.4 18.5 18.6 18.7 18.8 18.9 19.  19.1 19.2 19.3
 19.4 19.6 19.7 19.8 19.9 20.  20.1 20.2 20.3 20.4 20.5 20.6 20.7 20.8
 20.9 2